Data Cleaning Web Scraped Job Data
In approaching the task of cleaning my data, I had a few objectives:
1. eliminate all duplicate job postings from my data
2. format all available salary data so that salaries were presented in terms of a single figure detailing expected annual compensation
3. conduct any initial cleaning of string data that will make future natural language processing easier when building my predictive models

https://towardsdatascience.com/data-cleaning-web-scraped-job-data-6c2a2d963cd

In [2]:
import pandas as pd
#reading csv with index_col = 0, otherwise I get an additional
#unnamed column of separate index values
scrape_data = pd.read_csv("datascience.csv", index_col=0)
scrape_data.shape
#dropping any duplicate rows:
scrape_data = scrape_data.drop_duplicates()
scrape_data.reset_index(drop=True, inplace=True)
scrape_data.shape

(1506, 6)

In [3]:
scrape_data["salary"].value_counts()

Nothing_found    1506
Name: salary, dtype: int64

In [ ]:
#some targeted cleaning of salary information to make parsing easier #—> remove “\n”, “$”, and “,”
scrape_data["salary"] = scrape_data["salary"].str.replace("\n", "")
scrape_data["salary"] = scrape_data["salary"].str.replace(",", "")
scrape_data["salary"] = scrape_data["salary"].str.replace("$", "")

In [ ]:
scrape_data["og_salary_period"] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
scrape_data.ix[scrape_data["salary"].str.contains("year"), "og_salary_period"] = "year"
scrape_data.ix[scrape_data["salary"].str.contains("month"), "og_salary_period"] = "month"
scrape_data.ix[scrape_data["salary"].str.contains("week"), "og_salary_period"] = "week"
scrape_data.ix[scrape_data["salary"].str.contains("day"), "og_salary_period"] = "day"
scrape_data.ix[scrape_data["salary"].str.contains("hour"), "og_salary_period"] = "hour"

In [ ]:
#filtering out the results with salary data != "Nothing_found"
salary_data = scrape_data[scrape_data["salary"] != "Nothing_found"]
#removing all rows in salary data from scrape data, and converting #all "Nothing_found" values to NaN, so that float salary values can #be easily reintegrated later
scrape_data = scrape_data[~scrape_data.isin(salary_data)].dropna(how="all")
scrape_data["salary"].replace("Nothing_found",np.nan, inplace=True)
scrape_data["salary"].astype("float")

In [ ]:
#separating out data based on whether contains hour, day, week, #month, year
year_salaries = salary_data[salary_data["salary"].str.contains("year")]
month_salaries = salary_data[salary_data["salary"].str.contains("month")]
week_salaries = salary_data[salary_data["salary"].str.contains("week")]
day_salaries = salary_data[salary_data["salary"].str.contains("day")]
hour_salaries = salary_data[salary_data["salary"].str.contains("hour")]
# removing string values(" a year", " a week", etc. from salary dfs)
year_salaries["salary"] = year_salaries["salary"].str.replace(" a year", "")
month_salaries["salary"] = month_salaries["salary"].str.replace(" a month", "")
week_salaries["salary"] = week_salaries["salary"].str.replace(" a week", "")
day_salaries["salary"] = day_salaries["salary"].str.replace(" a day", "")
hour_salaries["salary"] = hour_salaries["salary"].str.replace(" an hour", "")

In [ ]:
def split_sal(i):
 try:
   splt = i.split(" — ",1)
   first = float(splt[0])
   second = float(splt[1])
   return (first + second)/2
 except:
   return float(i)

In [ ]:
year_salaries[“salary”] = year_salaries[“salary”].apply(split_sal)
month_salaries[“salary”] = month_salaries[“salary”].apply(split_sal)
month_salaries[“salary”] = month_salaries[“salary”] * 12
week_salaries[“salary”] = week_salaries[“salary”].apply(split_sal)
week_salaries[“salary”] = week_salaries[“salary”] * 52
day_salaries[“salary”] = day_salaries[“salary”].apply(split_sal)
day_salaries[“salary”] = day_salaries[“salary”] * 260
hour_salaries[“salary”] = hour_salaries[“salary”].apply(split_sal)
hour_salaries[“salary”] = hour_salaries[“salary”] * 2080

In [ ]:
#rejoining salary data into main scrape_data df
combined_salaries = pd.concat([year_salaries, month_salaries, week_salaries, day_salaries, hour_salaries], axis=0)
scrape_data = pd.concat([scrape_data, combined_salaries], axis=0)

In [ ]:
#targeted replace of “r&d” in job titles and summaries with #“research development”, as I want to remove “&” 
#from listings as part of general clean-up
scrape_data[“job_title”] = scrape_data[“job_title”].str.replace(“R&D”, “research development”)
scrape_data[“summary”] = scrape_data[“summary”].str.replace(“R&D”, “research development”)

In [ ]:
#creating data character cleaning function, and applying to all #columns, also lowercasing all string data for ease of later nlp
def data_clean(df, column):
  cleaning_list = [“+”, “$”,”/”,”,”,”?”,”.”,”;”,”-”,”@”,”!”,”&”,”%”,”^”,”*”,”)”,”(“, “\n”]
  for item in cleaning_list:
    df[column] = df[column].str.replace(item, “ “)
    df[column] = map(str.lower, df[column])
#can’t clean the salary column due to float values, and don’t need #to clean og_salary, so keeping out of the for loop
for column in scrape_data.columns[0:len(scrape_data.columns)-2]:
  data_clean(scrape_data, column)

In [ ]:
#where location == “united states” I filled this in with the city #location name, otherwise it’ll be useless if more than one city has #location data of “united states”
#using numpy is faster than using .replace
scrape_data[‘location’] = np.where(scrape_data[‘location’] == “united states”, scrape_data[“city”], scrape_data[‘location’])
#I’m also preemptively simplfying location data to include only city #and state, cutting data off at zip code - note: “ \d” splits #information on the first numeric digit in the string.
scrape_data[“location”] = scrape_data[“location”].str.split(‘ \d’, expand=True, n=1)
scrape_data.tail()

In [ ]:
scrape_data.to_csv(“scraped_clean.csv”)